In [ ]:
from pyfinite import ffield
import numpy as np
import pandas as pd
import math
import random 
F=ffield.FField(7)
encrypted_message = 2##<-- This is where we will put our converted string. 
elliptic_curve_value = 0
Encryption_point = [encrypted_message, 0]
Used_Point = [0, 0]
Quadratic_curve_constant = 0
scalar_curve_constant = 0
##Ok, we need to select the curve: 
for k in range(1, 128):
    for i in range(0, 10):
        for j in range(0, 10):
            a = F.Multiply(F.Multiply(encrypted_message, encrypted_message), encrypted_message)
            b = F.Multiply(i, F.Multiply(encrypted_message, encrypted_message))
            c = j
            if(F.Add(F.Add(F.Multiply(k, k), F.Multiply(encrypted_message, k)), F.Add(F.Add(a, b), c) % (131)) < 1 and i < 10 and j < 10):
                Encryption_point[1] = k 
                Quadratic_curve_constant = i
                scalar_curve_constant = j
                break
print(Encryption_point, Quadratic_curve_constant, scalar_curve_constant)
##Ok, now we need to select the Used_Point:
for i in range(10, 128):
    for j in range(10, 128):
        a = F.Multiply(F.Multiply(i, i), i)
        if(F.Add(F.Add(F.Multiply(j, j), F.Multiply(i, j)), F.Add(F.Add(a, Quadratic_curve_constant), scalar_curve_constant) % (131)) < 1 and i > 10 and j > 10):
            Used_Point[0] = i
            Used_Point[1] = j
print("Used Point:", Used_Point)
def Scalar_Add(pointa, pointb, elliptic_curve_value):
    pointa = np.array(pointa)
    pointb = np.array(pointb)
    xa=pointa[0]
    ya=pointa[1]
    xb=pointb[0]
    yb=pointb[1] 
    if((pointa == pointb).all()):
        lambdavalue = F.Add(xa, F.Multiply(ya, F.Inverse(xa)))
        xnew = F.Add(F.Add(F.Multiply(lambdavalue, lambdavalue), lambdavalue),elliptic_curve_value)
        ynew = F.Add(F.Multiply(xa, xa), F.Multiply(F.Add(1, lambdavalue), xnew))
    else: 
        lambdavalue = F.Multiply(F.Add(ya, yb), F.Inverse(F.Add(xa, xb)))
        xnew = F.Add(F.Add(F.Add(F.Add(F.Multiply(lambdavalue, lambdavalue), lambdavalue), elliptic_curve_value), xa), xb)
        ynew = F.Add(F.Add(F.Multiply(lambdavalue, F.Add(xa, xnew)), xnew), ya)
    return [xnew, ynew] 
def negation(pointc):
    pointc[1] = F.Add(pointc[0], pointc[1]) 
    return pointc
def Scalar_Multiplication(n, pointx, elliptic_curve_value): 
    Digits = [int(d) for d in str(bin(n))[2:]]
    initial = np.array([0, 0])
    for i in reversed(Digits):
        if(i == 1 and (initial == np.array([0, 0])).all()):
            initial = np.add(initial, pointx)
        elif (i == 1):
            initial = Scalar_Add(initial, pointx, elliptic_curve_value)
        pointx = Scalar_Add(pointx, pointx, elliptic_curve_value)
    return initial
public_key = Scalar_Multiplication(97, Used_Point, Quadratic_curve_constant)
print("Public Key:",public_key)
c1 = Scalar_Add(Encryption_point ,Scalar_Multiplication(49, public_key , Quadratic_curve_constant), Quadratic_curve_constant)
print("c1:",c1)
c2 = Scalar_Multiplication(49, Used_Point, Quadratic_curve_constant)
print("c2:",c2)
##c1 and c2 are what we need for encryption, the next line is decryption
print(Scalar_Add(c1, negation(Scalar_Multiplication(97, c2, Quadratic_curve_constant)), Quadratic_curve_constant))


In [2]:
from pyfinite import ffield
import numpy as np
import pandas as pd
import math
import random
F=ffield.FField(100)
def Scalar_Add(pointa, pointb, elliptic_curve_value):
    pointa = np.array(pointa)
    pointb = np.array(pointb)
    xa=int(pointa[0])
    ya=int(pointa[1])
    xb=int(pointb[0])
    yb=int(pointb[1])
    if((pointa == pointb).all()):
        lambdavalue = F.Add(xa, F.Multiply(ya, F.Inverse(xa)))
        xnew = F.Add(F.Add(F.Multiply(lambdavalue, lambdavalue), lambdavalue),elliptic_curve_value)
        ynew = F.Add(F.Multiply(xa, xa), F.Multiply(int(F.Add(1, lambdavalue)), int(xnew)))
    else: 
        lambdavalue = F.Multiply(F.Add(ya, yb), F.Inverse(F.Add(xa, xb)))
        xnew = F.Add(F.Add(F.Add(F.Add(F.Multiply(lambdavalue, lambdavalue), lambdavalue), elliptic_curve_value), xa), xb)
        ynew = F.Add(F.Add(F.Multiply(lambdavalue, F.Add(xa, xnew)), xnew), ya)
    return [xnew, ynew] 
def negation(pointc):
    pointc[1] = F.Add(pointc[0], pointc[1]) 
    return pointc
def Scalar_Multiplication(n, pointx, elliptic_curve_value): 
    Digits = [int(d) for d in str(bin(n))[2:]]
    initial = np.array([0, 0])
    for i in reversed(Digits):
        if(i == 1 and (initial == np.array([0, 0])).all()):
            initial = np.add(initial, pointx)
        elif (i == 1):
            initial = Scalar_Add(initial, pointx, elliptic_curve_value)
        pointx = Scalar_Add(pointx, pointx, elliptic_curve_value)
    return initial

def encryption(n):
    encrypted_message = n ##<-- This is where we will put our converted string. 
    elliptic_curve_value = 0
    Encryption_point = [encrypted_message, 0]
    Used_Point = [0, 0]
    scalar_curve_constant = 0
##Ok, we need to select the curve: 
    for k in range(1, 256):
        for i in range(0, random.randint(10, 256)):
            for j in range(0, random.randint(10, 256)):
                a = F.Multiply(F.Multiply(encrypted_message, encrypted_message), encrypted_message)
                b = F.Multiply(i, F.Multiply(encrypted_message, encrypted_message))
                c = j
                if(F.Add(F.Add(F.Multiply(k, k), F.Multiply(encrypted_message, k)), F.Add(F.Add(a, b), c) % (131)) < 1):
                    Encryption_point[1] = k 
                    Quadratic_curve_constant = i
                    scalar_curve_constant = j
                    break
##Ok, now we need to select the Used_Point:
    for i in range(10, 128):
        for j in range(10, 128):
            a = F.Multiply(F.Multiply(i, i), i)
            if(F.Add(F.Add(F.Multiply(j, j), F.Multiply(i, j)), F.Add(F.Add(a, Quadratic_curve_constant), scalar_curve_constant) % (131)) < 1 and i > 10 and j > 10):
                Used_Point[0] = i
                Used_Point[1] = j
                break
    a = random.randint(int(2**50), int(2**100))
    public_key = Scalar_Multiplication(a, Used_Point, Quadratic_curve_constant)
    c1 = Scalar_Add(Encryption_point ,Scalar_Multiplication(int(2**107-1), public_key, Quadratic_curve_constant), Quadratic_curve_constant)
    c2 = Scalar_Multiplication(int(2**107-1), Used_Point, Quadratic_curve_constant)
    return list(c1 + c2 + [Quadratic_curve_constant] + [a])

def decryption(a, b, Quadratic_curve_constant, private_key):
    return Scalar_Add(a, negation(Scalar_Multiplication(private_key, b, Quadratic_curve_constant)), Quadratic_curve_constant)

print(decryption([372030926261995512726108777234, 150035582309307103445320937903], [59212243314813666660759037824, 1054131204347016327369497865405], 197,360485422838994333057978306964))

[19, 23]
